# Spatial Join



In [2]:
import os
import time
import cupy
import cudf
import cuspatial
import numpy as np

## Data

* All `.cny` files can be downloaded from http://geoteci.engr.ccny.cuny.edu/nyctaxidata/
* The binary files are `int32` type in the unit of feet using projection epsg 2236 (for NYC and Long Island) each record has 4 fields (pick_up_lon, pick_up_lat,drop_off_lon, drop_off_lat). Only the first 2 fields are used the orginal 2009 yellow cab data can be accessed from https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page.
* The taxi zone data in ESRI shapefile format can be accessed from https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page which is linked to https://s3.amazonaws.com/nyc-tlc/misc/taxi_zones.zip; the shapefile also uses EPSG 2263 projection.
* The NYC Community District Data (nycd) and 2000 Census Tract data (nyct2000) can be downloaded from https://www1.nyc.gov/site/planning/data-maps/open-data.page#district_political version 11a released in 2011 were used; both use EPSG 2263 projection.

In [3]:
cuspatial_data_path='data' 
polygon_shapefile_name='taxi_zones.shp'
#polygon_shapefile_name='nycd_11a_av/nycd.shp'
#polygon_shapefile_name='nyct2000_11a_av/nyct2000.shp'

def read_points(path):
    print('reading points file:', path)
    points = np.fromfile(path, dtype=np.int32)
    points = cupy.asarray(points)
    points = points.reshape((len(points)// 4, 4))
    points = cudf.DataFrame(points)
    return points

#use 6 month data for RTX 2080 with 8GB memory 
points_df = cudf.concat(
    [read_points(os.path.join(cuspatial_data_path,
        '2009{}.cny'.format('0{}'.format(i) if i < 10 else i)))
    for i in range(1, 12)]).reset_index(drop=True)
points_df['x'] = points_df[0].astype(np.float32)
points_df['y'] = points_df[1].astype(np.float32)
print(len(points_df))

reading points file: data/200901.cny
reading points file: data/200902.cny
reading points file: data/200903.cny
reading points file: data/200904.cny
reading points file: data/200905.cny
reading points file: data/200906.cny
reading points file: data/200907.cny
reading points file: data/200908.cny
reading points file: data/200909.cny
reading points file: data/200910.cny
reading points file: data/200911.cny
95879703


## Quadtree construction

Spatial indexing is similar to relational database indexing that uses B-Tree or alike. Here quadtree is used to index points.

In [5]:
ply_fpos, ply_rpos, ply_vertices = cuspatial.read_polygon_shapefile(
    os.path.join(cuspatial_data_path , polygon_shapefile_name)) 
x1,x2,y1,y2 =(ply_vertices['x'].min(), ply_vertices['x'].max(), 
    ply_vertices['y'].min(),  ply_vertices['y'].max())
                                 
num_levels = 15
min_size = 512
scale = max(abs(x2 - x1), abs(y2 - y1)) / ((1 << num_levels) - 2);

start = time.time()
key_to_point,quadtree = cuspatial.quadtree_on_points(points_df['x'],points_df['y'],
    x1,x2,y1,y2, scale,num_levels, min_size)
end = time.time()

print('number of nodes:',len(quadtree))
print('node count:', (quadtree['is_quad'] == True).sum())
print('leaf count:', (quadtree['is_quad'] == False).sum())
print('quadtree construction time :', (end-start)*1000)

number of nodes: 560184
node count: 140090
leaf count: 420094
quadtree construction time : 60.14585494995117


## Spatial filtering

The idea of spatial filtering is to pair up point quadrants and polygon bounding boxes if they spatially overlap. Those that do not overlap are filtered out (like screening).

In [6]:
poly_bboxes = cuspatial.polygon_bounding_boxes(
    ply_fpos, ply_rpos, ply_vertices['x'], ply_vertices['y'])
print(len(poly_bboxes))    

start = time.time()
intersections = cuspatial.join_quadtree_and_bounding_boxes(
    quadtree, poly_bboxes,x1,x2,y1,y2, scale, num_levels,
)
end = time.time()
print('number of polygon-quadrant pairs:',len(intersections))    
print('spatial filtering time :', (end-start)*1000)

263
number of polygon-quadrant pairs: 933665
spatial filtering time : 12.319326400756836


## Spatial refinement

Spatial refinement is to test whether a point covered by a quadrant is actually in the polygon represented by its bounding box.

In [7]:
start = time.time()
polygons_and_points = cuspatial.quadtree_point_in_polygon(
    intersections, quadtree,key_to_point,
    points_df['x'],points_df['y'],
    ply_fpos,ply_rpos,ply_vertices['x'],ply_vertices['y'])
end = time.time()
print(len(polygons_and_points)) 
print('spatial refinement time :', (end-start)*1000)

95238407
spatial refinement time : 495.99337577819824
